In [1]:
import pandas as pd
import os

os.chdir("../data/tickers/ticker_data")
print(os.getcwd())
columns = ["adjClose", "adjHigh", "adjLow", "adjOpen", "adjVolume", "close", "date", "divCash", "high", "low", "open", "splitFactor", "volume"]

c:\Users\anton\Documents\antoniouaa\msc_thesis\data\tickers\ticker_data


In [4]:
df = pd.read_csv("002287.csv", parse_dates=True, header=0)
df

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,2018-01-01T00:00:00.000Z,33.88,33.88,33.88,33.88,0,32.939316,32.939316,32.939316,32.939316,0,0.0,1.0
1,2018-01-02T00:00:00.000Z,34.65,34.78,33.78,33.80,770080,33.687937,33.814327,32.842093,32.861537,770080,0.0,1.0
2,2018-01-03T00:00:00.000Z,35.26,35.68,34.40,34.83,995280,34.281000,34.689339,33.444878,33.862939,995280,0.0,1.0
3,2018-01-04T00:00:00.000Z,34.96,35.33,34.86,35.16,666590,33.989330,34.349057,33.892106,34.183777,666590,0.0,1.0
4,2018-01-05T00:00:00.000Z,35.04,35.24,34.69,34.69,473460,34.067108,34.261555,33.726826,33.726826,473460,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2018-11-26T00:00:00.000Z,25.81,26.20,25.16,25.40,682120,25.446809,25.831321,24.805956,25.042578,682120,0.0,1.0
236,2018-11-27T00:00:00.000Z,26.20,26.46,25.82,26.06,470680,25.831321,26.087662,25.456668,25.693291,470680,0.0,1.0
237,2018-11-28T00:00:00.000Z,26.39,26.80,26.00,26.32,380700,26.018647,26.422878,25.634135,25.949632,380700,0.0,1.0
238,2018-11-29T00:00:00.000Z,26.60,27.40,26.39,26.39,788370,26.225692,27.014435,26.018647,26.018647,788370,0.0,1.0


In [7]:
df.describe()

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
count,240.000000,240.000000,240.000000,240.000000,2.400000e+02,240.000000,240.000000,240.000000,240.000000,2.400000e+02,240.000000,240.0
mean,29.706375,30.184917,29.280333,29.722583,8.906588e+05,29.039016,29.506512,28.622685,29.054802,8.906588e+05,0.001667,1.0
std,3.348591,3.428705,3.278655,3.339583,9.351730e+05,3.125547,3.199746,3.059889,3.115767,9.351730e+05,0.025820,0.0
min,22.330000,22.720000,21.430000,22.320000,0.000000e+00,22.015778,22.400291,21.128443,22.005919,0.000000e+00,0.000000,1.0
25%,27.152500,27.400000,26.990000,27.160000,4.241925e+05,26.718656,27.014435,26.516541,26.740840,4.241925e+05,0.000000,1.0
50%,29.100000,29.430000,28.705000,29.145000,6.469600e+05,28.552483,28.887699,28.083003,28.553647,6.469600e+05,0.000000,1.0
75%,32.405000,32.857500,31.825000,32.345000,1.069432e+06,31.505270,31.945206,30.941374,31.446936,1.069432e+06,0.000000,1.0
max,37.200000,38.800000,36.100000,37.600000,9.641170e+06,36.167136,37.722711,35.097677,36.556030,9.641170e+06,0.400000,1.0
